In [1]:
import os


os.environ["LD_LIBRARY_PATH"] = "/home/jclee/CoppeliaSim/libcoppeliaSim.so.1"




import openai # 0.28.1
from arguments import get_config # config 파일 불러와서 활용할 용도.
from interfaces import setup_LMP
from visualizers import ValueMapVisualizer
from envs.rlbench_env import VoxPoserRLBench
from utils import set_lmp_objects
import numpy as np
from rlbench import tasks

import os
from dotenv import load_dotenv

from logger import Logger

load_dotenv()
# .env 에 기재된 OPENAI_API_KEY가 os.environ에 환경변수로 저장.
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", None)


openai.api_key = OPENAI_API_KEY

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Setup

In [2]:
config = get_config('rlbench')
# uncomment this if you'd like to change the language model (e.g., for faster speed or lower cost)
# for lmp_name, cfg in config['lmp_config']['lmps'].items():
#     cfg['model'] = 'gpt-3.5-turbo'

# initialize env and voxposer ui
visualizer = ValueMapVisualizer(config['visualizer'])
env = VoxPoserRLBench(visualizer=visualizer)
lmps, lmp_env = setup_LMP(env, config, debug=False)
voxposer_ui = lmps['plan_ui']

##################################################
## voxel resolution: [0.0105 0.0131 0.01  ]
##################################################




## Playground

By default we use one of the instructions that come with each task. However, you may treat each task as simply a scene initialization from RLBench, and feel free to try any task you can come up with for the scene.

Note:
- Whether an instruction can be executed or not depends on 1) whether relevant objects are available, and 2) capabilities of the overall algorithm.
- Each execution may produce one or more visualizations. You may view them in "./visualizations/" folder.
- The prompts are adapted with minimal change from the real-world environment in the VoxPoser paper. If a task failure is due to incorrect code generated by the LLM, feel free to modify the relevant prompt in "./prompts/" folder.
- You may view the reward by printing "env.latest_reward". These are computed by RLBench for each task.
- To inspect in viewer without performing any action, you may call "env.rlbench_env._scene.step()" in a loop.

In [3]:
# # uncomment this to show all available tasks in rlbench
# # NOTE: in order to run a new task, you need to add the list of objects (and their corresponding env names) to the "task_object_names.json" file. See README for more details.
# print([task for task in dir(tasks) if task[0].isupper() and not '_' in task])

In [4]:
# below are the tasks that have object names added to the "task_object_names.json" file
# uncomment one to use
# env.load_task(tasks.PutRubbishInBin)
# env.load_task(tasks.LampOff)
# env.load_task(tasks.OpenWineBottle)
# env.load_task(tasks.PushButton)
# env.load_task(tasks.TakeOffWeighingScales)
# env.load_task(tasks.MeatOffGrill)
# env.load_task(tasks.SlideBlockToTarget)
# env.load_task(tasks.TakeLidOffSaucepan)
env.load_task(tasks.TakeUmbrellaOutOfUmbrellaStand)

descriptions, obs = env.reset()
set_lmp_objects(lmps, env.get_object_names())  # set the object names to be used by voxposer

In [6]:
log_filename="log_CaP/tasks.TakeUmbrellaOutOfUmbrellaStand"

logger = Logger(log_filename)

instruction = np.random.choice(descriptions)
print(f"Executing instruction: {instruction}")

voxposer_ui(instruction)

logger.close()
print(f"log_file : {log_filename} is stored.")

In [8]:
env.rlbench_env.shutdown()
